# In vehicle coupon recommendation 

## Table of contents
- [1 - Packages](#1) 
- [2 - Explornatory Data Analysis](#2)
- [3 - Feature Enigineering](#3)
- [4 - Classification Models](#4)
- [5 - Clustering](#5)

# 0 Load Package

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from sklearn.feature_selection import chi2
from scipy import stats
from pandas import read_csv
from sklearn.model_selection import train_test_split

# 1 Data Preprocessing

## 1.1 Know your data

### 1.1.1 Show the first 5 records

In [ ]:
df = pd.read_csv("D:\Data-Science\Course\IE7374\Projects\ml_project\Vehicle_Coupon.csv")
df.head()
df.describe()

### 1.1.2 Dimension

In [ ]:
df.shape

### 1.1.3 Features Explanations

### 1.1.4 Features Statistics

In [ ]:
df.info()

In [ ]:
df.describe()

### 1.1.5 Null or not

In [ ]:
df.isnull().sum() / df.shape[0] * 100

## 1.2 Data Cleaning and transformation

### 1.2.1 Drop car column

In [ ]:
df.drop(columns='car', inplace=True)
# df.columns

### 1.2.2 Set data type as category

In [ ]:
df = df.astype('category')
df.dtypes

### 1.2.2 Impute lack data

[Ref](https://jamesrledoux.com/code/imputation#:~:text=One%20approach%20to%20imputing%20categorical,given%20in%20Pandas'%20value_counts%20function.)

In [ ]:
df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))
df.isnull().sum().sum()

## 2 Exploratory Data Analysis

## 2.1 Coupon acception vs. reject

In [ ]:
def make_pie(sizes, text,colors,labels):
    import matplotlib.pyplot as plt
    import numpy as np

    col = [[i/255. for i in c] for c in colors]

    fig, ax = plt.subplots()
    ax.axis('equal')
    width = 0.35
    kwargs = dict(colors=col, startangle=180, autopct='%1.1f%%')
    outside, _, _ = ax.pie(sizes, radius=1, pctdistance=1-width/2,
                       labels=labels,**kwargs)
    plt.setp( outside, width=width, edgecolor='white')

    kwargs = dict(size=20, fontweight='bold', va='center')
    ax.text(0, 0, text, ha='center', **kwargs)
    plt.show()

c1 = (226,33,7)
c2 = (60,121,189)

make_pie([257,90], "Acception",[c1,c2],['Y','N'])


## 2.2 Which feature will be indepedent of target?

[Ref](https://medium.com/analytics-vidhya/constructing-heat-map-for-chi-square-test-of-independence-6d78aa2b140f#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjFiZDY4NWY1ZThmYzYyZDc1ODcwNWMxZWIwZThhNzUyNGM0NzU5NzUiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2NTc1NzcwNTcsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjExNjk2NjQyODk4MDIxNTU1MjE0OSIsImVtYWlsIjoieWFubWluZ2xpdTIxQGdtYWlsLmNvbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJuYW1lIjoiWWFubWluZyBMaXUiLCJwaWN0dXJlIjoiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EvQUl0YnZtbW1DdnlEVzktaWF0dXQzYXhlQWFlZGpIWV9TY2laNWI4M3dfLTY9czk2LWMiLCJnaXZlbl9uYW1lIjoiWWFubWluZyIsImZhbWlseV9uYW1lIjoiTGl1IiwiaWF0IjoxNjU3NTc3MzU3LCJleHAiOjE2NTc1ODA5NTcsImp0aSI6IjhmMmJjM2MwOTk2NDZmMDdiODBhYTZmNWE5OGY5ZGU0YTk3YzAxMTQifQ.Tvz3S-XbT029APxcmsPETDBUj8S1ir0iYejfv9QbpWLtuCDYwxxbDuQv1FCL9Wz_D8hmxPutvuOgrF-dFT5oMYTq6rn0hV8Z0UNNn8rBR_KofiuSx10ChpZgJfmMAtqccsfW20Yk__v_xvZPNOLnjiHdXZAtoQiWUNo9VFAZ58NM98KjLfkkJ0r715iuIQrnyzWnUnabgwO96GT1OTaLynipdg6ugvClNENa3WZIncBtdqlP4jeTE7QOEPnTMRUntHGCTj5VRsnJRYt3OYpE6BqQgANptFUygI-K47IM5WSAicSFzmxyp9CgJxQ3U4d4S3LPKqiBy63v7gkxkLJPhQ)

In [ ]:
df = df.astype('object')
column_names=df.columns

chisqmatrix=pd.DataFrame(df,columns=column_names,index=column_names)


outercnt=0
innercnt=0
for icol in column_names:
    for jcol in column_names:
       mycrosstab=pd.crosstab(df[icol],df[jcol])
       stat,p,dof,expected=stats.chi2_contingency(mycrosstab)
       chisqmatrix.iloc[outercnt,innercnt]=round(p,3)
       cntexpected=expected[expected<5].size
       perexpected=((expected.size-cntexpected)/expected.size)*100
       if perexpected<20:
            chisqmatrix.iloc[outercnt,innercnt]=2
       if icol==jcol:
           chisqmatrix.iloc[outercnt,innercnt]=0.00
       innercnt=innercnt+1
    outercnt=outercnt+1
    innercnt=0
    

plt.figure(figsize = (16,12))
sns.heatmap(chisqmatrix.astype(np.float64), annot=True,linewidths=0.1, 
            cmap='coolwarm')

# Question: 2 perspectives here. If we see the bottom row of this heatmap, 
# we can easily remove those with bigger p values with alpha = .05 for they 
# are indepdent. How do we deal those variables among features for a relatie
# small value?

## 2.3 How do the rest of feature will influence the target ?

[Ref](https://stackoverflow.com/questions/63687789/how-do-i-create-a-pie-chart-using-categorical-data-in-matplotlib)

In [ ]:
count = 0
target = df.columns[-1]
for cat_val in df.columns:
  if cat_val == target: 
    continue
  temp_df = df.loc[:, [cat_val, target]]  
  temp_df = temp_df.groupby([cat_val, target]).size().reset_index(name='count')
  pivot_df = pd.pivot_table(temp_df, values='count', index=cat_val, columns=target)
  pivot_df.plot(kind='bar')
  count += 1

<a name='3'></a>
## 3 - Feature Engineering

<a name='3-1'></a>
### 3.1 -  Feature selection
We assume that the target and features are related only the confidence is above 95 percent. By observing the bottom row in the heatmap in section 2.2, those 3 columns, **toCoupon_GEQ5min, direction_same, direction_opp**, are removed as not meeting the confidence conditions.

In [ ]:
df_droped = df.drop(columns = ['toCoupon_GEQ5min', 'direction_same', 'direction_opp'])
print(len(df.columns))
print(len(df_droped.columns))
df_droped.to_csv('Clean_Data_0.csv')

<a name=3-2></a>
### 3.2 - Feature encoding

In [ ]:
for i in df_droped.columns:
    print(i, df_droped[i].unique(), '\n')

#### 3.2.1 - Load dataset 

Numpy

In [ ]:
def load_dataset(filename):
    data = read_csv(filename, header=None)
    dataset = data.values
    
    X = dataset[:, :-1]
    y = dataset[:,-1]
    
    X = X.astype(str)
    y = y.reshape((len(y), 1))
    return X, y

In [ ]:
# Test 
X, y = load_dataset('Clean_Data_0.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train

Pandas

In [ ]:
df_droped.head()

In [ ]:
target = ['Y']
XP = df_droped.drop(columns = target)
yP = df_droped[target]
XTrain, XTest, yTrain, yTest = train_test_split(XP, yP, test_size=0.3, random_state=0)
XTrain.head()

<a name='3-2-2'></a>
#### 3.2.2 - Ordinal encoding

In [ ]:
# %pip install category_encoders
import category_encoders as ce

[Encoding categorical variables](https://kiwidamien.github.io/encoding-categorical-variables.html)\
[Target Encoding](https://medium.com/analytics-vidhya/target-encoding-vs-one-hot-encoding-with-simple-examples-276a7e7b3e64)

In [ ]:
def ordinal_encoder(train, test, columns):
    oe = ce.OrdinalEncoder(cols=columns, return_df=True)
    train_transformed = oe.fit_transform(train)
    test_transformed = oe.transform(test)
    return train_transformed, test_transformed

In [ ]:
print(df_droped.columns)
ordial_columns = ['age', 'education', 'income', 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']
XTrain_after_ordinal, XTest_after_ordinal = ordinal_encoder(XTrain, XTest, ordial_columns)
display(XTrain.head())
display(XTest.head())
display(XTrain_after_ordinal.head())
display(XTest_after_ordinal.head())

# Question: Are time, temprature the ordinal variables?

In [ ]:
def ordinal_encoding(clean_df):
    frequency_map = {
        'never': 0,
        'less1': 1,
        '1~3': 2,
        '4~8': 3,
        'gt8': 4
    }
    
    age_map = {
        'below21': 0,
        '21': 1,
        '26': 2,
        '31': 3,
        '36': 4,
        '41': 5,
        '46': 6,
        '50plus': 7
    }
    
    income_map = {
        'Less than $12500': 0,
        '$12500 - $24999': 1,
        '$25000 - $37499': 2,
        '$37500 - $49999': 3,
        '$50000 - $62499': 4,
        '$62500 - $74999': 5,
        '$75000 - $87499': 6,
        '$87500 - $99999': 7,
        '$100000 or More': 8
    }
    
    frequency_cols = ['Restaurant20To50', 'RestaurantLessThan20',
                      'CarryAway', 'CoffeeHouse', 'Bar']

    
    education_map = {
        'Some High School': 0, 
        'High School Graduate': 1,
        'Some college - no degree': 2,
        'Associates degree': 3,
        'Bachelors degree': 4,
        'Graduate degree (Masters or Doctorate)': 5
    }
        
    
    for col in frequency_cols:
        clean_df[col] = clean_df[col].map(frequency_map)
    clean_df.age = clean_df.age.map(age_map)
    clean_df.income = clean_df.income.map(income_map)
    clean_df.education = clean_df.education.map(education_map)

    return clean_df

In [ ]:
df_droped.education.unique()

In [ ]:
df_droped.head()

<a name = '3-2-2'></a>
#### 3.2.2 - One hot encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder


def one_hot_encoding(train, test, columns):
    oe = ce.OneHotEncoder(cols=columns, return_df=True)
    train_transformed = oe.fit_transform(train)
    test_transformed = oe.transform(test)
    return train_transformed, test_transformed


In [ ]:
train_after_one_hot_encoding, test_after_one_hot_encoding = one_hot_encoding(XTrain, XTest, columns=[i for i in XTrain.columns if i not in ordial_columns])
display(train_after_one_hot_encoding.head())
display(test_after_one_hot_encoding.head())
display(train_after_one_hot_encoding.age)



<a name = '3-2-3'></a>
#### 3.2.3 - Frequency and target encoding

In [ ]:
def target_encoding(train, ytrain, test, columns):
    ytrain = ytrain.astype(float)
    oe = ce.TargetEncoder(return_df=True)

    train_transformed = oe.fit_transform(train[columns], ytrain[ytrain.columns])
    test_transformed = oe.transform(test)
    return train_transformed, test_transformed

In [ ]:
train_after_target_encoding, test_after_target_encoding = target_encoding(XTrain, yTrain, XTest, columns=list(XTrain.columns))
display(train_after_target_encoding.head())
display(test_after_target_encoding.head())

In [ ]:
# Test cases
import random

from numpy import disp
random.seed(4)
data=pd.DataFrame({'class':['A','B','C','B','C','A','A','A'],
                    'yes':['1','2','3','4','5','6','7','8'],
                    'Marks':[random.randint(0,1) for i in range(8)]})
encoder=ce.TargetEncoder(cols='class') 
display(data)
encoder.fit_transform(data['class'],data['Marks'])

X = data.drop(columns=['Marks'])
y = data[['Marks']]
display(X)
display(y)
x1, x2, y1, y2 = train_test_split(X, y, test_size=0.3, random_state=0)
display(pd.concat([x1, y1], axis=1))
display(pd.concat([x2, y2], axis=1))
display(x1)

t1, t2 = target_encoding(x1, y1, x2, columns=list(x1.columns))
display(t1)
display(t2)

<a name = '3-2-4'></a>
#### 3.2.4 - Embedded encoding

[Ref](https://machinelearningmastery.com/how-to-prepare-categorical-data-for-deep-learning-in-python/)

In [ ]:
from numpy import unique
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import concatenate
from keras.utils import plot_model
 
# load the dataset
def load_dataset(filename):
	# load the dataset as a pandas DataFrame
	data = read_csv(filename, header=0).iloc[:,1:]
	# retrieve numpy array
	dataset = data.values
	# split into input (X) and output (y) variables
	X = dataset[:, :-1]
	y = dataset[:,-1]
	# format all fields as string
	X = X.astype(str)
	# reshape target to be a 2d array
	y = y.reshape((len(y), 1))
	return X, y
 
# prepare input data
def prepare_inputs(X_train, X_test):
	X_train_enc, X_test_enc = list(), list()
	# label encode each column
	for i in range(X_train.shape[1]):
		le = LabelEncoder()
		le.fit(X_train[:, i])
		# encode
		train_enc = le.transform(X_train[:, i])
		test_enc = le.transform(X_test[:, i])
		# store
		X_train_enc.append(train_enc)
		X_test_enc.append(test_enc)
	return X_train_enc, X_test_enc
 
# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc
 
# load the dataset
X_e, y_e = load_dataset('Clean_Data_0.csv')
display(X_e, y_e)
# split into train and test sets
X_Train, X_Test, y_Train, y_Test = train_test_split(X_e, y_e, test_size=0.3, random_state=0)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_Train, X_Test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_Train, y_Test)
# make output 3d
y_train_enc = y_train_enc.reshape((len(y_train_enc), 1, 1))
y_test_enc = y_test_enc.reshape((len(y_test_enc), 1, 1))

# prepare each input head
in_layers = list()
em_layers = list()
for i in range(len(X_train_enc)):
	# calculate the number of unique inputs
	n_labels = len(unique(X_train_enc[i]))
	# define input layer
	in_layer = Input(shape=(1,))
	# define embedding layer
	em_layer = Embedding(n_labels, 10)(in_layer)
	# store layers
	in_layers.append(in_layer)
	em_layers.append(em_layer)
# concat all embeddings
merge = concatenate(em_layers)
dense = Dense(10, activation='relu', kernel_initializer='he_normal')(merge)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=in_layers, outputs=output)
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# plot graph
plot_model(model, show_shapes=True, to_file='embeddings.png')
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=20, batch_size=16, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
display(merge)

<a name = '4'></a>
# 4 - Classifiaction Models

<a name = '4-1'></a>
### 4.1 - Logistic regression


In [ ]:
# Target Encoding
# %pip install dmba
from sklearn.linear_model import LogisticRegression
from dmba import classificationSummary, gainsChart, liftChart

display(train_after_target_encoding)
lr = LogisticRegression()
yTrain = yTrain.astype('category')
yTest = yTest.astype('category')
lr.fit(train_after_target_encoding, yTrain)
classificationSummary(yTest, lr.predict(test_after_target_encoding))

In [ ]:
# One Hot Encoding 
XTrain_only_one_hot, XTest_only_one_hot = one_hot_encoding(XTrain, XTest, columns=XTrain.columns)
display(XTrain_only_one_hot.head())
lr.fit(XTrain_only_one_hot, yTrain)
classificationSummary(yTest, lr.predict(XTest_only_one_hot))

In [ ]:
# Ordinal Encoding 
XTrain_only_ordinal, XTest_only_ordinal = ordinal_encoder(XTrain, XTest, columns=XTrain.columns)
display(XTrain_only_ordinal.head())
lr.fit(XTrain_only_ordinal, yTrain)
classificationSummary(yTest, lr.predict(XTest_only_ordinal))

In [ ]:
# One Hot Encoding + Ordianal Encoding
XTrain_after_ordinal, XTest_after_ordinal = ordinal_encoder(XTrain, XTest, ordial_columns)
train_after_one_hot_encoding, test_after_one_hot_encoding = one_hot_encoding(XTrain_after_ordinal, XTest_after_ordinal, columns=[i for i in XTrain.columns if i not in ordial_columns])
display(train_after_one_hot_encoding.head())
display(test_after_one_hot_encoding)

lr.fit(train_after_one_hot_encoding, yTrain)
classificationSummary(yTest, lr.predict(test_after_one_hot_encoding))


## Ordinal encoding with some order assignment
target = ['Y']
XP = df_droped.drop(columns = target)
yP = df_droped[target]
XTrain, XTest, yTrain, yTest = train_test_split(XP, yP, test_size=0.3, random_state=0)
display(XTrain.head())

XTrain_after_ordinal_o = ordinal_encoding(XTrain)
XTest_after_ordinal_o = ordinal_encoding(XTest)
display(XTrain.head())
display(XTrain_after_ordinal_o.isnull().sum())
display(XTest_after_ordinal_o.isnull().sum())
train_after_one_hot_encoding_n, test_after_one_hot_encoding_n = one_hot_encoding(XTrain_after_ordinal_o, XTest_after_ordinal_o, columns=[i for i in XTrain.columns if i not in ordial_columns])
display(train_after_one_hot_encoding_n.head())
display(test_after_one_hot_encoding_n)

lr.fit(train_after_one_hot_encoding_n, yTrain)
classificationSummary(yTest, lr.predict(test_after_one_hot_encoding_n))

In [ ]:
# Embedded Encoding
# We have tested previously using neural network, how to apply to logistic 
# regression here?
# It seems that the encoder has changed the dimension of the training data
# so I am not sure if they are applicable to the logistic regression.
display(len(em_layers))
em_layers

df_droped.shape[1]

### Deep Neural Network